# Переделала для разных текстов

In [44]:
!pip install pymorphy3
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import math
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

1. Применить к текстам лемматизацию, удаление стоп слов и токенизацию по словам

In [46]:
texts = [
    "Задача организации, в особенности же постоянное информационно-техническое обеспечение нашей деятельности требует определения и уточнения системы масштабного изменения ряда параметров.",
    "Практический опыт показывает, что начало повседневной работы по формированию позиции представляет собой интересный эксперимент проверки системы обучения кадров, соответствующей насущным потребностям!",
    "Задача организации, в особенности же дальнейшее развитие различных форм деятельности напрямую зависит от всесторонне сбалансированных нововведений?"
]

morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def preprocessing(texts: list[str]) -> list[list[str]]:
  preprocessing_texts = []
  for text in texts:
    tokens = word_tokenize(text.lower())
    lemmatized_words = [
      morph.parse(token)[0].normal_form
      for token in tokens
      if token not in stop_words]

    preprocessing_texts.append(lemmatized_words)
  return preprocessing_texts

preprocessed_texts = preprocessing(texts)
print(preprocessed_texts)

[['задача', 'организация', ',', 'особенность', 'постоянный', 'информационно-технический', 'обеспечение', 'наш', 'деятельность', 'требовать', 'определение', 'уточнение', 'система', 'масштабный', 'изменение', 'ряд', 'параметр', '.'], ['практический', 'опыт', 'показывать', ',', 'начало', 'повседневный', 'работа', 'формирование', 'позиция', 'представлять', 'себя', 'интересный', 'эксперимент', 'проверка', 'система', 'обучение', 'кадр', ',', 'соответствовать', 'насущный', 'потребность', '!'], ['задача', 'организация', ',', 'особенность', 'дальнейший', 'развитие', 'различный', 'форма', 'деятельность', 'напрямую', 'зависеть', 'всесторонне', 'сбалансированный', 'нововведение', '?']]


2. Реализовать Bag of Words

In [47]:
def make_dict(texts: list[str]) -> dict[str]:
  word_freq = {}

  for i, words in enumerate(preprocessed_texts):
      for word in words:
          if word not in word_freq:
              word_freq[word] = {}
          if f'text_{i}' not in word_freq[word]:
              word_freq[word][f'text_{i}'] = 0
          word_freq[word][f'text_{i}'] += 1

  return word_freq

BoW = make_dict(texts)
print(BoW)

{'задача': {'text_0': 1, 'text_2': 1}, 'организация': {'text_0': 1, 'text_2': 1}, ',': {'text_0': 1, 'text_1': 2, 'text_2': 1}, 'особенность': {'text_0': 1, 'text_2': 1}, 'постоянный': {'text_0': 1}, 'информационно-технический': {'text_0': 1}, 'обеспечение': {'text_0': 1}, 'наш': {'text_0': 1}, 'деятельность': {'text_0': 1, 'text_2': 1}, 'требовать': {'text_0': 1}, 'определение': {'text_0': 1}, 'уточнение': {'text_0': 1}, 'система': {'text_0': 1, 'text_1': 1}, 'масштабный': {'text_0': 1}, 'изменение': {'text_0': 1}, 'ряд': {'text_0': 1}, 'параметр': {'text_0': 1}, '.': {'text_0': 1}, 'практический': {'text_1': 1}, 'опыт': {'text_1': 1}, 'показывать': {'text_1': 1}, 'начало': {'text_1': 1}, 'повседневный': {'text_1': 1}, 'работа': {'text_1': 1}, 'формирование': {'text_1': 1}, 'позиция': {'text_1': 1}, 'представлять': {'text_1': 1}, 'себя': {'text_1': 1}, 'интересный': {'text_1': 1}, 'эксперимент': {'text_1': 1}, 'проверка': {'text_1': 1}, 'обучение': {'text_1': 1}, 'кадр': {'text_1': 1}

3. Реализовать TF-IDF

In [48]:
def count_num_words(texts: list[str]) -> list[int]:
    num_words = []
    for text in texts:
        num_words.append(len(word_tokenize(text)))
    return num_words
def compute_tf(BoW: dict[str], texts: list[str]) -> dict[str]:
    tf = {}
    num_words = count_num_words(texts)

    for word, freq in BoW.items():
        tf[word] = {}
        for doc, count in freq.items():
            doc_index = int(doc.split('_')[1])
            tf[word][doc] = count / num_words[doc_index]

    return tf

tf = compute_tf(BoW, texts)
print(tf)

{'задача': {'text_0': 0.047619047619047616, 'text_2': 0.05555555555555555}, 'организация': {'text_0': 0.047619047619047616, 'text_2': 0.05555555555555555}, ',': {'text_0': 0.047619047619047616, 'text_1': 0.08333333333333333, 'text_2': 0.05555555555555555}, 'особенность': {'text_0': 0.047619047619047616, 'text_2': 0.05555555555555555}, 'постоянный': {'text_0': 0.047619047619047616}, 'информационно-технический': {'text_0': 0.047619047619047616}, 'обеспечение': {'text_0': 0.047619047619047616}, 'наш': {'text_0': 0.047619047619047616}, 'деятельность': {'text_0': 0.047619047619047616, 'text_2': 0.05555555555555555}, 'требовать': {'text_0': 0.047619047619047616}, 'определение': {'text_0': 0.047619047619047616}, 'уточнение': {'text_0': 0.047619047619047616}, 'система': {'text_0': 0.047619047619047616, 'text_1': 0.041666666666666664}, 'масштабный': {'text_0': 0.047619047619047616}, 'изменение': {'text_0': 0.047619047619047616}, 'ряд': {'text_0': 0.047619047619047616}, 'параметр': {'text_0': 0.

In [49]:
def compute_idf(BoW: dict[str], num_docs: int) -> dict[str]:
    idf = {}

    for word in BoW:
        idf[word] = math.log(num_docs / len(BoW[word]))
    return idf

idf = compute_idf(BoW, len(texts))
print(idf)

{'задача': 0.4054651081081644, 'организация': 0.4054651081081644, ',': 0.0, 'особенность': 0.4054651081081644, 'постоянный': 1.0986122886681098, 'информационно-технический': 1.0986122886681098, 'обеспечение': 1.0986122886681098, 'наш': 1.0986122886681098, 'деятельность': 0.4054651081081644, 'требовать': 1.0986122886681098, 'определение': 1.0986122886681098, 'уточнение': 1.0986122886681098, 'система': 0.4054651081081644, 'масштабный': 1.0986122886681098, 'изменение': 1.0986122886681098, 'ряд': 1.0986122886681098, 'параметр': 1.0986122886681098, '.': 1.0986122886681098, 'практический': 1.0986122886681098, 'опыт': 1.0986122886681098, 'показывать': 1.0986122886681098, 'начало': 1.0986122886681098, 'повседневный': 1.0986122886681098, 'работа': 1.0986122886681098, 'формирование': 1.0986122886681098, 'позиция': 1.0986122886681098, 'представлять': 1.0986122886681098, 'себя': 1.0986122886681098, 'интересный': 1.0986122886681098, 'эксперимент': 1.0986122886681098, 'проверка': 1.0986122886681098,

In [51]:

def compute_tfidf(tf: dict[str], idf: dict[str]) -> dict[str]:
    tfidf = {}

    for word in tf:
        tfidf[word] = {}
        for doc in tf[word]:
            tfidf[word][doc] = tf[word][doc] * idf[word]

    return tfidf

tfidf = compute_tfidf(tf, idf)
print(tfidf)

{'задача': {'text_0': 0.01930786229086497, 'text_2': 0.022525839339342466}, 'организация': {'text_0': 0.01930786229086497, 'text_2': 0.022525839339342466}, ',': {'text_0': 0.0, 'text_1': 0.0, 'text_2': 0.0}, 'особенность': {'text_0': 0.01930786229086497, 'text_2': 0.022525839339342466}, 'постоянный': {'text_0': 0.05231487088895761}, 'информационно-технический': {'text_0': 0.05231487088895761}, 'обеспечение': {'text_0': 0.05231487088895761}, 'наш': {'text_0': 0.05231487088895761}, 'деятельность': {'text_0': 0.01930786229086497, 'text_2': 0.022525839339342466}, 'требовать': {'text_0': 0.05231487088895761}, 'определение': {'text_0': 0.05231487088895761}, 'уточнение': {'text_0': 0.05231487088895761}, 'система': {'text_0': 0.01930786229086497, 'text_1': 0.016894379504506847}, 'масштабный': {'text_0': 0.05231487088895761}, 'изменение': {'text_0': 0.05231487088895761}, 'ряд': {'text_0': 0.05231487088895761}, 'параметр': {'text_0': 0.05231487088895761}, '.': {'text_0': 0.05231487088895761}, 'п